In [46]:
"""
[ 사용자 집단별 추천 ]
- 비슷한 특성의 사람들을 소집단으로 만들고 각 집단의 평점평균 바탕으로 추천
- 소집단 만드는 기준 example) 성별, 직업, 나이
"""

'\n[ 사용자 집단별 추천 ]\n- 비슷한 특성의 사람들을 소집단으로 만들고 각 집단의 평점평균 바탕으로 추천\n- 소집단 만드는 기준 example) 성별, 직업, 나이\n'

In [47]:
# Data 읽어오기
from google.colab import files
users=files.upload()

Saving user.txt to user (1).txt


In [66]:
# User Data
import io
import pandas as pd
u_cols=['user_id','age','sex','occupation','zip_code']
users=pd.read_csv('user.txt',sep='|',names=u_cols)
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [49]:
# u.item file을 Dataframe으로 읽기 (영화 정보 )
data=files.upload()

Saving u.item to u (1).item


In [50]:
cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventure','Animation','children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romancce','Sci-Fi','Thriller','War','Western']
movies=pd.read_csv('u.item',sep='|',names=cols,encoding='latin=1')
#movies=movies.set_index('movie_id')
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romancce,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [51]:
# u.data file을 DataFrame으로 읽기
data1=files.upload()

Saving u.data to u (1).data


In [52]:
r_cols=['user_id','movie_id','rating','timestamp']
ratings=pd.read_csv('u.data',sep='\t',names=r_cols,encoding='latin=1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [53]:
# timestamp 제거
ratings=ratings.drop('timestamp',axis=1)

In [54]:
# movie id, title 빼고 다른 데이터 전부 제거
movies=movies[['movie_id','title']]

In [55]:
"""
[ train set/ test set 으로 분리하기 ]
- 분리하지 않으면 정확도의 왜곡 생김
"""

'\n[ train set/ test set 으로 분리하기 ]\n- 분리하지 않으면 정확도의 왜곡 생김\n'

In [56]:
from sklearn.model_selection import train_test_split # sklearn_selection library 이용해 데이터 분리
x=ratings.copy() # ratings data 복사
y=ratings['user_id'] # user_id 기준으로 train set, test set 분리
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25, stratify=y) # 75%-> train set(x_train), 25% -> test set(x_test), y_train & y_test -> user id저장   

In [57]:
# 정확도 계산하는 Function
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

In [58]:
"""
Score: 예측 model을 넘겨받아서 해당 모델로 예측한 예측치의 정확도(RMSE) 계산
"""

'\nScore: 예측 model을 넘겨받아서 해당 모델로 예측한 예측치의 정확도(RMSE) 계산\n'

In [67]:
import numpy as np
# Model 별 RMSE 계산하는 Function 
def score(model):
  id_pairs=zip(x_test['user_id'],x_test['movie_id'])  # test set의 user_id, movie_id를 pair로 짝맞춰 data만듬
  y_pred=np.array([model(user,movie) for (user,movie) in id_pairs]) # 예측모델에 의해 (user,movie)에 대한 예측값 계산해 list로 저장
  y_true=np.array(x_test['rating']) # 실제 평점값 list를 x_true에서 받아오기
  return RMSE(y_true, y_pred) # 예측값 list & 실제값 list에 대한 RMSE구하기

In [68]:
# train data로 Full Matrix구하기 -> 추천 알고리즘에 사용
rating_matrix=x_train.pivot(index='user_id',columns='movie_id',values='rating') 
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1632,1633,1634,1636,1637,1639,1641,1642,1643,1644,1645,1646,1647,1648,1649,1651,1652,1655,1656,1657,1658,1660,1661,1662,1664,1665,1666,1667,1669,1670,1671,1672,1673,1674,1676,1677,1678,1679,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,3.0,NaN,NaN,4.0,1.0,NaN,3.0,NaN,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,NaN,NaN,4.0,NaN,3.0,NaN,3.0,2.0,4.0,NaN,3.0,NaN,5.0,NaN,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
"""
사용자 특성에 따라 집단 나눠서 예측하려면 평점 데이터 (u.data) + 사용자 데이터(u.user) 필요
=> 2개의 data를 merge => 평점 data의 full matrix구하기

* 모든 사용자가 영화에 대해서 평가하지는 않았으므로 NaN 존재
"""

'\n사용자 특성에 따라 집단 나눠서 예측하려면 평점 데이터 (u.data) + 사용자 데이터(u.user) 필요\n=> 2개의 data를 merge => 평점 data의 full matrix구하기\n\n* 모든 사용자가 영화에 대해서 평가하지는 않았으므로 NaN 존재\n'

In [69]:
# best-seller Model 수정
def best_seller(user_id,movie_id):
  try:
    rating=train_mean[movie_id] # 해당 영화가 train_mean에 존재하면 평균값 
  except:
    rating=3.0 # 존재하지 X -> 기본값(3.0)
  return rating

train_mean=x_train.groupby(['movie_id'])['rating'].mean() # train set에 대한 평균 구하기
score(best_seller) # best-seller방식으로 예측한 경우에 대한 RMSE


1.0279371657462386

In [70]:
"""
* try- except사용 이유?
train set에는 없는데 test set에 존재하는 영화로 인해 발생하는 오류 방지
"""

'\n* try- except사용 이유?\ntrain set에는 없는데 test set에 존재하는 영화로 인해 발생하는 오류 방지\n'

In [71]:
# Full matrix(x_train)을 user data와 merge
merged_ratings=pd.merge(x_train,users) # merge function: 공통 이름의 key를 기준으로 합침 -> user_id기준으로 merge
users=users.set_index('user_id') # user의 index를 user_id로 정함. 나중에 데이터 확인

In [72]:
merged_ratings

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,627,62,4,24,M,engineer,08034
1,627,528,4,24,M,engineer,08034
2,627,53,4,24,M,engineer,08034
3,627,949,2,24,M,engineer,08034
4,627,724,2,24,M,engineer,08034
...,...,...,...,...,...,...,...
74995,740,1038,4,25,F,educator,22206
74996,740,289,4,25,F,educator,22206
74997,740,258,3,25,F,educator,22206
74998,740,322,3,25,F,educator,22206


In [73]:
#gender별 평점평균 계산
"""
1. merged_raings에서 3가지 정보만 남김 
2. movie_id, sex기준으로 'rating'의 평균 구해서 저장
"""
g_mean=merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean()
g_mean


movie_id  sex
1         F      3.804598
          M      3.934783
2         F      3.333333
          M      3.238636
3         F      2.785714
                   ...   
1677      F      3.000000
1678      M      1.000000
1679      M      3.000000
1680      M      2.000000
1681      M      3.000000
Name: rating, Length: 3039, dtype: float64

In [74]:
# gender 기준 추천
# gender별 평균을 예측치로 돌려주는 Function
def cf_gender(user_id,movie_id):
  if movie_id in rating_matrix: # 예측치 구하는 영화가 데이터에 있는지?
    gender=users.loc[user_id]['sex'] # user의 성별 가져오기  
    if gender in g_mean[movie_id]: # 예측대상 영화가 성별 평균인지 (평가한 사람이 적으면 gender_mean이 없을 수도 있음) 
      gender_rating=g_mean[movie_id][gender] # 평균값의 예측치
    else:
      gender_rating=3.0
  else:
    gender_rating=3.0
  return gender_rating
score(cf_gender)

1.0348093637645506

In [ ]:
"""
gender기준으로 영화 추천 시 값이 약 1.034
best-seller방식으로 영화 추천 시 RMSE값 : 1.025
로 성별에 따른 추천이 정확도를 개선하지는 못함을 알 수 있다  
"""

'\ngender기준으로 영화 추천 시 값이 약 1.034\nbest-seller방식으로 영화 추천 시 RMSE값 : 1.025\n로 성별에 따른 추천이 정확도를 개선하지는 못함을 알 수 있다  \n'

In [75]:
"""
연습문제 2-1 ) 사용자 직업에 따라 집단을 나눠에 예측값 구하는 함수 만들고 정확도 계산
"""
# occupation별 평점 평균 계산
occu_mean=merged_ratings[['movie_id','occupation','rating']].groupby(['movie_id','occupation'])['rating'].mean()
occu_mean

movie_id  occupation   
1         administrator    3.850000
          artist           4.375000
          doctor           3.333333
          educator         3.730769
          engineer         4.027027
                             ...   
1677      student          3.000000
1678      student          1.000000
1679      student          3.000000
1680      student          2.000000
1681      writer           3.000000
Name: rating, Length: 16783, dtype: float64

In [76]:
# occupation 기준 추천
# occupation별 영화 점수 평균을 예측치로 돌려주는 Fuction
def cf_occu(user_id,movie_id):
  if movie_id in rating_matrix:
    occupation=users.loc[user_id]['occupation']
    if occupation in occu_mean[movie_id]:
      occu_rating=occu_mean[movie_id][occupation]
    else:
      occu_rating=3.0
  else:
    occu_rating=3.0
  return occu_rating
score(cf_occu)

1.1168294463167603

In [78]:
"""
연습문제 2-2 ) 사용자의 성별과 직업을 동시에 고려한 집단을 나우어서 예측값을 구하는 함수 만들고 정확도 계산
"""

#gender, occupation별 평점 평균 계산
both_mean=merged_ratings[['movie_id','sex','occupation','rating']].groupby(['movie_id','occupation','sex'])['rating'].mean()
both_mean

movie_id  occupation     sex
1         administrator  F      3.916667
                         M      3.750000
          artist         F      4.500000
                         M      4.250000
          doctor         M      3.333333
                                  ...   
1677      student        F      3.000000
1678      student        M      1.000000
1679      student        M      3.000000
1680      student        M      2.000000
1681      writer         M      3.000000
Name: rating, Length: 22523, dtype: float64

In [79]:
# 성별, 직업 고려해서 추천
# occupation, sex별 영화 점수 평균을 예측치로 돌려주는 Function
def cf_both(user_id,movie_id):
  if movie_id in rating_matrix:
    b_occupation=users.loc[user_id]['occupation']
    b_gender=users.loc[user_id]['sex']
    if b_occupation and b_gender in both_mean[movie_id]:
      both_rating=both_mean[movie_id][b_occupation][b_gender]
    else:
      both_rating=3.0
  else:
    both_rating=3.0
  return both_rating
score(cf_both)   

1.2485992151206888

In [ ]:
"""
성별, 직업을 모두 고려하는 경우 RMSE값이 1.25로 다른 방식들 보다 제일 높은 정확도를 보인다.
"""